# Vector Store / VectorDB

## FAISS

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("speech.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 30)
docs = text_splitter.split_documents(documents)

In [3]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

In [5]:
embeddings = OllamaEmbeddings(model = 'nomic-embed-text')
db = FAISS.from_documents(docs, embeddings)
db

In [7]:
# querrying
querry = "What is the speaker saying about the war?"
docs = db.similarity_search(querry)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

### Retriever

In [8]:
retriever = db.as_retriever()
retriever.invoke(querry)

[Document(metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foun

### Similarity Search with Scores

In [9]:
docs_score = db.similarity_search_with_score(querry)
docs_score

[(Document(metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  316.7571),
 (Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon 

### embeddings

In [10]:
embedded_vectors = embeddings.embed_query(querry)
embedded_vectors

[0.029168125241994858,
 1.0612292289733887,
 -4.222771644592285,
 -1.2837541103363037,
 2.25410795211792,
 2.305576801300049,
 -0.9816261529922485,
 0.10263475775718689,
 0.70070481300354,
 -0.05193329229950905,
 -0.05703829973936081,
 1.0687484741210938,
 0.0034555718302726746,
 1.6035531759262085,
 1.6412955522537231,
 -0.939726710319519,
 -0.8219319581985474,
 -1.2511454820632935,
 -1.1249275207519531,
 0.6633350849151611,
 -0.7864494323730469,
 -0.3666594624519348,
 -0.43878689408302307,
 0.12129860371351242,
 2.3580689430236816,
 0.7144963145256042,
 -0.31621283292770386,
 0.9844256639480591,
 -0.8020786643028259,
 -0.45306605100631714,
 1.6705224514007568,
 -0.9582518339157104,
 -0.4542257785797119,
 0.2954939007759094,
 -1.0589348077774048,
 -0.7231097221374512,
 0.15969547629356384,
 0.5916778445243835,
 0.29284703731536865,
 -0.7188343405723572,
 0.31553584337234497,
 -0.2856482267379761,
 0.419980525970459,
 -2.019214391708374,
 1.5437467098236084,
 -0.5113024115562439,
 -1.1

In [11]:
doc_vec = db.similarity_search_by_vector(embedded_vectors)
doc_vec

[Document(metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foun

In [12]:
db.save_local("faiss_local_index")

In [14]:
new_df = FAISS.load_local("faiss_local_index", embeddings, allow_dangerous_deserialization=True)

## ChromaDB

In [20]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [21]:
loader = TextLoader("speech.txt")
data = loader.load()
text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 30)
splits = text_splitter.split_documents(data)

In [24]:
embeddings = OllamaEmbeddings()
vectordb = Chroma.from_documents(documents=splits, embedding=embeddings)
vectordb

TypeError: 'type' object is not subscriptable